# Read results

In [1]:
#read raw results

import pandas as pd
from available_models import available_models
import os
import numpy as np


def read_score(folder, kind, slide, overlap):
    filename = f"{folder}/iou.{slide}.{overlap}.txt" if kind == 'model' else f"{folder}/iou.postproc.{slide}.{overlap}.txt"
    if os.path.exists(filename):
        try:
            with open(filename, "r") as f:
                return float(f.read())
        except:
            print(f"failed to read {filename}")
    else:
        # print(f"Not found: {filename}")
        pass

    return None
            

def get_results():
    slides = ["alpha", "beta", "gamma"]

    overlaps = [25, 50]
    rows = []    
    for level, tubule_area, color_mode, generation in list(available_models.keys()):
        model_name = available_models[(level, tubule_area, color_mode, generation)]
        folder = f"output/{model_name}"
        row = {
            "level": level,
            "tubule area": tubule_area,
            "color mode": color_mode,
            "generation": generation,
            "model":f"{level}:{tubule_area}:{color_mode}",
            "date": model_name[-15:]
        }

        for s in slides:
            for o in overlaps:            
                row[f"model iou {s} {o}"] = read_score(folder, 'model', s, o)
                row[f"postp iou {s} {o}"] = read_score(folder, 'postproc', s, o)

        rows.append(row)


    return pd.DataFrame(rows).set_index("model")

    

df = get_results()
df

,level,tubule area,color mode,generation,date,model iou alpha 25,postp iou alpha 25,model iou alpha 50,postp iou alpha 50,model iou beta 25,postp iou beta 25,model iou beta 50,postp iou beta 50,model iou gamma 25,postp iou gamma 25,model iou gamma 50,postp iou gamma 50
model,,,,,,,,,,,,,,,,,
1:1:color,1,1,color,0,2023-04-23_1025,0.532,0.947,0.542,0.947,0.513,0.773,0.516,0.769,0.592,0.914,0.595,0.914
1:10:color,1,10,color,0,2023-03-01_0300,0.522,0.946,0.534,0.946,0.488,0.760,0.493,0.758,0.569,0.912,0.571,0.913
1:30:color,1,30,color,0,2023-04-22_1551,0.558,0.946,0.569,0.949,0.520,0.751,0.523,0.749,0.496,0.912,0.521,0.914
1:1:grayscale,1,1,grayscale,0,2023-04-24_2309,0.519,0.943,0.524,0.944,0.435,0.721,0.437,0.742,0.538,0.909,0.533,0.911
1:10:grayscale,1,10,grayscale,0,2023-04-24_1450,0.495,0.942,0.513,0.944,0.423,0.762,0.436,0.765,0.530,0.906,0.541,0.907
1:30:grayscale,1,30,grayscale,0,2023-04-24_2310,0.543,0.943,0.547,0.947,0.514,0.738,0.519,0.733,0.490,0.911,0.484,NaN
2:1:color,2,1,color,0,2023-04-23_1941,0.419,0.952,0.427,0.952,0.418,0.637,0.419,0.637,0.544,0.901,0.549,0.901
2:10:color,2,10,color,0,2023-03-02_0348,0.551,0.951,0.563,0.950,0.536,0.665,0.540,0.667,0.601,0.908,0.602,0.908
2:30:color,2,30,color,0,2023-04-23_0106,0.528,0.945,0.543,0.947,0.502,0.685,0.504,0.685,0.580,0.903,0.582,0.904


## Split into model and postprocessing outputs, join 3 test runs into one line

In [3]:
# one model per row


def get_pp_anb_model_dfs(df):
    

    levels = list(df.level.unique())                   # 1,2
    tubule_areas = list(df["tubule area"].unique())   # 1,10,30
    color_modes = list(df["color mode"].unique())    # ["color", "grayscale"]
    slides = ["alpha", "beta", "gamma"]
    overlaps = [25, 50]
    generations = [0,1,2]

    post_rows =[]
    modl_rows = []
    modl_row=None
    for level in levels:
        for tubule_area in tubule_areas:
            for color_mode in color_modes:
                post_row = {"level": level, "tubule_area": tubule_area, "color_mode": color_mode}
                modl_row = {"level": level, "tubule_area": tubule_area, "color_mode": color_mode}
                for s in slides:
                    for o in overlaps:

                        data = df.loc[f"{level}:{tubule_area}:{color_mode}"]
                        post_values = list(data[f"postp iou {s} {o}"])
                        modl_values = list(data[f"model iou {s} {o}"])
                        for i in generations:
                            post_row[f"{s} {o} {i}"] = post_values[i]
                            modl_row[f"{s} {o} {i}"] = modl_values[i]

                post_rows.append(post_row)
                modl_rows.append(modl_row)

    post_df = pd.DataFrame(post_rows)
    modl_df = pd.DataFrame(modl_rows)
    
    return post_df, modl_df


post_df, modl_df = get_pp_anb_model_dfs(df)

In [6]:
post_df

,level,tubule_area,color_mode,alpha 25 0,alpha 25 1,alpha 25 2,alpha 50 0,alpha 50 1,alpha 50 2,beta 25 0,...,beta 25 2,beta 50 0,beta 50 1,beta 50 2,gamma 25 0,gamma 25 1,gamma 25 2,gamma 50 0,gamma 50 1,gamma 50 2
0,1,1,color,0.947,0.954,0.943,0.947,0.955,0.944,0.773,...,0.746,0.769,0.747,0.748,0.914,0.912,0.912,0.914,0.912,0.913
1,1,1,grayscale,0.943,0.927,0.941,0.944,0.928,0.945,0.721,...,0.703,0.742,0.693,0.699,0.909,0.908,0.909,0.911,0.910,0.911
2,1,10,color,0.946,0.942,0.950,0.946,0.943,0.953,0.760,...,0.772,0.758,0.696,0.771,0.912,0.910,0.912,0.913,0.909,0.913
3,1,10,grayscale,0.942,0.942,0.939,0.944,0.945,0.945,0.762,...,0.771,0.765,0.775,0.782,0.906,0.906,0.908,0.907,0.906,0.910
4,1,30,color,0.946,0.948,0.948,0.949,0.945,0.944,0.751,...,0.734,0.749,0.743,0.732,0.912,0.910,0.910,0.914,0.910,0.911
5,1,30,grayscale,0.943,0.945,0.937,0.947,0.946,0.942,0.738,...,0.725,0.733,0.754,0.720,0.911,0.907,0.903,NaN,0.909,0.905
6,2,1,color,0.952,0.938,0.952,0.952,0.940,0.953,0.637,...,0.717,0.637,0.649,0.711,0.901,0.898,0.904,0.901,0.898,0.904
7,2,1,grayscale,0.947,0.952,0.942,0.947,0.954,0.945,0.676,...,0.633,0.677,0.660,0.633,0.901,0.909,0.907,0.902,0.909,0.908
8,2,10,color,0.951,0.945,0.955,0.950,0.946,0.956,0.665,...,0.685,0.667,0.650,0.685,0.908,0.905,0.909,0.908,0.905,0.910
9,2,10,grayscale,0.939,0.952,0.954,0.938,0.953,0.953,0.641,...,0.661,0.642,0.656,0.662,0.902,0.906,0.906,0.904,0.906,0.906


# Means and std deviation

In [9]:
def get_means(df):
    variants=["25 0", "25 1", "25 2", "50 0", "50 1", "50 2"]

    alpha_cols = [f"alpha {v}" for v in variants]
    beta_cols = [f"beta {v}" for v in variants]
    gamma_cols = [f"gamma {v}" for v in variants]


    df["mean_alpha"] = df[alpha_cols].mean(axis=1).round(3)
    df["std_alpha"] = df[alpha_cols].std(axis=1).round(3)

    df["mean_beta"] = df[beta_cols].mean(axis=1).round(3)
    df["std_beta"] = df[beta_cols].std(axis=1).round(3)

    df["mean_gamma"] = df[gamma_cols].mean(axis=1).round(3)
    df["std_gamma"] = df[gamma_cols].std(axis=1).round(3)

    return df[["level","tubule_area", "color_mode", "mean_alpha", "std_alpha", "mean_beta", "std_beta", "mean_gamma","std_gamma"]]


In [10]:
get_means(post_df)

,level,tubule_area,color_mode,mean_alpha,std_alpha,mean_beta,std_beta,mean_gamma,std_gamma
0,1,1,color,0.948,0.005,0.755,0.012,0.913,0.001
1,1,1,grayscale,0.938,0.008,0.704,0.025,0.910,0.001
2,1,10,color,0.947,0.004,0.746,0.031,0.912,0.002
3,1,10,grayscale,0.943,0.002,0.770,0.007,0.907,0.002
4,1,30,color,0.947,0.002,0.741,0.008,0.911,0.002
5,1,30,grayscale,0.943,0.004,0.732,0.012,0.907,0.003
6,2,1,color,0.948,0.007,0.667,0.037,0.901,0.003
7,2,1,grayscale,0.948,0.004,0.656,0.020,0.906,0.004
8,2,10,color,0.950,0.005,0.667,0.016,0.908,0.002
9,2,10,grayscale,0.948,0.008,0.653,0.009,0.905,0.002


In [11]:
get_means(modl_df)

,level,tubule_area,color_mode,mean_alpha,std_alpha,mean_beta,std_beta,mean_gamma,std_gamma
0,1,1,color,0.514,0.018,0.504,0.009,0.586,0.006
1,1,1,grayscale,0.482,0.051,0.388,0.058,0.529,0.029
2,1,10,color,0.500,0.036,0.478,0.013,0.569,0.013
3,1,10,grayscale,0.526,0.041,0.438,0.018,0.548,0.020
4,1,30,color,0.519,0.063,0.463,0.051,0.496,0.068
5,1,30,grayscale,0.537,0.013,0.470,0.044,0.530,0.034
6,2,1,color,0.432,0.015,0.422,0.004,0.555,0.007
7,2,1,grayscale,0.505,0.085,0.457,0.065,0.575,0.055
8,2,10,color,0.530,0.023,0.527,0.020,0.596,0.009
9,2,10,grayscale,0.553,0.046,0.471,0.059,0.590,0.015


## Best configs slide by slide

### On postprocessed results

In [37]:
get_means(post_df).sort_values("mean_alpha", ascending=False).head()

,level,tubule_area,color_mode,mean_alpha,std_alpha,mean_beta,std_beta,mean_gamma,std_gamma
8,2,10,color,0.950,0.005,0.667,0.016,0.908,0.002
0,1,1,color,0.948,0.005,0.755,0.012,0.913,0.001
6,2,1,color,0.948,0.007,0.667,0.037,0.901,0.003
7,2,1,grayscale,0.948,0.004,0.656,0.020,0.906,0.004
9,2,10,grayscale,0.948,0.008,0.653,0.009,0.905,0.002


In [41]:
get_means(post_df).sort_values("mean_beta", ascending=False).head()

,level,tubule_area,color_mode,mean_alpha,std_alpha,mean_beta,std_beta,mean_gamma,std_gamma
3,1,10,grayscale,0.943,0.002,0.770,0.007,0.907,0.002
0,1,1,color,0.948,0.005,0.755,0.012,0.913,0.001
2,1,10,color,0.947,0.004,0.746,0.031,0.912,0.002
4,1,30,color,0.947,0.002,0.741,0.008,0.911,0.002
5,1,30,grayscale,0.943,0.004,0.732,0.012,0.907,0.003


In [39]:
get_means(post_df).sort_values("mean_gamma", ascending=False).head()

,level,tubule_area,color_mode,mean_alpha,std_alpha,mean_beta,std_beta,mean_gamma,std_gamma
0,1,1,color,0.948,0.005,0.755,0.012,0.913,0.001
2,1,10,color,0.947,0.004,0.746,0.031,0.912,0.002
4,1,30,color,0.947,0.002,0.741,0.008,0.911,0.002
1,1,1,grayscale,0.938,0.008,0.704,0.025,0.910,0.001
8,2,10,color,0.950,0.005,0.667,0.016,0.908,0.002


Level 1 seems to outperform. (level1,tubule-1,color) best overall

### On raw model results

In [42]:
get_means(modl_df).sort_values("mean_alpha", ascending=False).head()

,level,tubule_area,color_mode,mean_alpha,std_alpha,mean_beta,std_beta,mean_gamma,std_gamma
9,2,10,grayscale,0.553,0.046,0.471,0.059,0.590,0.015
5,1,30,grayscale,0.537,0.013,0.470,0.044,0.530,0.034
8,2,10,color,0.530,0.023,0.527,0.020,0.596,0.009
3,1,10,grayscale,0.526,0.041,0.438,0.018,0.548,0.020
10,2,30,color,0.521,0.022,0.473,0.024,0.575,0.006


In [44]:
get_means(modl_df).sort_values("mean_beta", ascending=False).head()

,level,tubule_area,color_mode,mean_alpha,std_alpha,mean_beta,std_beta,mean_gamma,std_gamma
8,2,10,color,0.530,0.023,0.527,0.020,0.596,0.009
0,1,1,color,0.514,0.018,0.504,0.009,0.586,0.006
2,1,10,color,0.500,0.036,0.478,0.013,0.569,0.013
10,2,30,color,0.521,0.022,0.473,0.024,0.575,0.006
9,2,10,grayscale,0.553,0.046,0.471,0.059,0.590,0.015


In [43]:
get_means(modl_df).sort_values("mean_gamma", ascending=False).head()

,level,tubule_area,color_mode,mean_alpha,std_alpha,mean_beta,std_beta,mean_gamma,std_gamma
8,2,10,color,0.530,0.023,0.527,0.020,0.596,0.009
9,2,10,grayscale,0.553,0.046,0.471,0.059,0.590,0.015
0,1,1,color,0.514,0.018,0.504,0.009,0.586,0.006
7,2,1,grayscale,0.505,0.085,0.457,0.065,0.575,0.055
10,2,30,color,0.521,0.022,0.473,0.024,0.575,0.006


(level 2, tubule area 10, color) looks best overall...top 1 in two slides and 3rd place on alpha

# T-tests

In [17]:
from scipy.stats import ttest_ind

#level 1 or 2?
import numpy as np
from typing import List, Tuple

def get_column_names(slide_name: str) -> List[str]:
    # overlaps 25 & 50, 3 runs each
    variants=["25 0", "25 1", "25 2", "50 0", "50 1", "50 2"]    
    return [f"{slide_name} {v}" for v in variants]


def calculate_t_tests(df: pd.DataFrame, a:str, a_values: List, b:str, b_values: List, c:str, c_values, threshold=0.1)->Tuple[pd.DataFrame, pd.DataFrame]:
    print(f"doing t-test by {a}\n")

    summaries = []
    results = []
    
    for slide in ['alpha', 'beta', 'gamma']:
        cols = get_column_names(slide)
        df_slide = df[['level', 'tubule_area', 'color_mode']+cols]

        scores_first = list(df_slide[ df_slide[a]==a_values[0]][cols].unstack().dropna())
        scores_second = list(df_slide[ df_slide[a]==a_values[1]][cols].unstack().dropna())

        summaries.append({
            "slide": slide,
            f"{a}={a_values[0]}":round(np.mean(scores_first), 3),
            f"{a}={a_values[1]}":round(np.mean(scores_second), 3),
            "p-value": round(ttest_ind(scores_first, scores_second, equal_var=False).pvalue, 3)
        })

        for b_val in b_values:
            for c_val in c_values:
                df_subset = df_slide[
                    (df_slide[b]==b_val)&(df_slide[c]==c_val)
                ]
                scores_first = list(df_subset[ df_subset[a]==a_values[0]][cols].unstack().dropna())
                scores_second = list(df_subset[ df_subset[a]==a_values[1]][cols].unstack().dropna())
                
                results.append({
                    "slide": slide,
                    b: b_val,
                    c: c_val,
                    f"{a}={a_values[0]}":round(np.mean(scores_first), 3),
                    f"{a}={a_values[1]}":round(np.mean(scores_second), 3),
                    "p-value": round(ttest_ind(scores_first, scores_second, equal_var=False).pvalue, 3)
                })
            
    

    summaries_df = pd.DataFrame(summaries).set_index(['slide'])
    summaries_df['significant'] = summaries_df['p-value']<threshold
    
    result_df = pd.DataFrame(results).set_index(['slide',b,c])
    result_df['significant'] = result_df['p-value']<threshold
    return summaries_df, result_df
    


## T-tests on postprocessed results

### Color or grayscale?

In [18]:
calculate_t_tests(post_df, 'color_mode', ['color','grayscale'], 'level', [1,2], 'tubule_area', [1, 10, 30])[0]

doing t-test by color_mode



,color_mode=color,color_mode=grayscale,p-value,significant
slide,,,,
alpha,0.947,0.944,0.019,True
beta,0.708,0.692,0.175,False
gamma,0.908,0.906,0.034,True


In [19]:
calculate_t_tests(post_df, 'color_mode', ['color','grayscale'], 'level', [1,2], 'tubule_area', [1, 10, 30])[1]

doing t-test by color_mode



color_mode=color  color_mode=grayscale  p-value  \
slide level tubule_area                                                    
alpha 1     1                       0.948                 0.938    0.030   
            10                      0.947                 0.943    0.086   
            30                      0.947                 0.943    0.084   
      2     1                       0.948                 0.948    1.000   
            10                      0.950                 0.948    0.532   
            30                      0.942                 0.942    1.000   
beta  1     1                       0.755                 0.704    0.003   
            10                      0.746                 0.770    0.116   
            30                      0.741                 0.732    0.174   
      2     1                       0.667                 0.656    0.570   
            10                      0.667                 0.653    0.092   
            30                      0.670                 0.636    0.001   
gamma 1     1                       0.913                 0.910    0.001   
            10                      0.912                 0.907    0.001   
            30                      0.911                 0.907    0.039   
      2     1                       0.901                 0.906    0.022   
            10                      0.908                 0.905    0.045   
            30                      0.904                 0.900    0.097   

                         significant  
slide level tubule_area               
alpha 1     1                   True  
            10                  True  
            30                  True  
      2     1                  False  
            10                 False  
            30                 False  
beta  1     1                   True  
            10                 False  
            30                 False  
      2     1                  False  
            10                  True  
            30                  True  
gamma 1     1                   True  
            10                  True  
            30                  True  
      2     1                   True  
            10                  True  
            30                  True

### Level 1 or 2? 

In [20]:
calculate_t_tests(post_df, 'level', [1,2], 'color_mode', ['color','grayscale'],  'tubule_area', [1, 10, 30])[0]

doing t-test by level



,level=1,level=2,p-value,significant
slide,,,,
alpha,0.944,0.947,0.115,False
beta,0.742,0.658,0.000,True
gamma,0.910,0.904,0.000,True


In [45]:
calculate_t_tests(post_df, 'level', [1,2], 'color_mode', ['color','grayscale'],  'tubule_area', [1, 10, 30])[1]

doing t-test by level



level=1  level=2  p-value  significant
slide color_mode tubule_area                                        
alpha color      1              0.948    0.948    0.889        False
                 10             0.947    0.950    0.158        False
                 30             0.947    0.942    0.267        False
      grayscale  1              0.938    0.948    0.034         True
                 10             0.943    0.948    0.148        False
                 30             0.943    0.942    0.666        False
beta  color      1              0.755    0.667    0.001         True
                 10             0.746    0.667    0.001         True
                 30             0.741    0.670    0.000         True
      grayscale  1              0.704    0.656    0.005         True
                 10             0.770    0.653    0.000         True
                 30             0.732    0.636    0.000         True
gamma color      1              0.913    0.901    0.000         True
                 10             0.912    0.908    0.004         True
                 30             0.911    0.904    0.000         True
      grayscale  1              0.910    0.906    0.054         True
                 10             0.907    0.905    0.045         True
                 30             0.907    0.900    0.020         True

### Tubule area 1, 10 or 30?

In [21]:
calculate_t_tests(post_df, 'tubule_area', [1, 10], 'level', [1,2], 'color_mode', ['color','grayscale'])[0]

doing t-test by tubule_area



,tubule_area=1,tubule_area=10,p-value,significant
slide,,,,
alpha,0.946,0.947,0.415,False
beta,0.696,0.709,0.359,False
gamma,0.907,0.908,0.727,False


In [22]:
calculate_t_tests(post_df, 'tubule_area', [1, 30], 'level', [1,2], 'color_mode', ['color','grayscale'])[0]

doing t-test by tubule_area



,tubule_area=1,tubule_area=30,p-value,significant
slide,,,,
alpha,0.946,0.944,0.335,False
beta,0.696,0.695,0.963,False
gamma,0.907,0.906,0.223,False


In [23]:
calculate_t_tests(post_df, 'tubule_area', [10, 30], 'level', [1,2], 'color_mode', ['color','grayscale'])[0]

doing t-test by tubule_area



,tubule_area=10,tubule_area=30,p-value,significant
slide,,,,
alpha,0.947,0.944,0.031,True
beta,0.709,0.695,0.338,False
gamma,0.908,0.906,0.073,True


## T-tests on raw model results

### Color or Grayscale?

In [25]:
calculate_t_tests(modl_df, 'color_mode', ['color','grayscale'], 'level', [1,2], 'tubule_area', [1, 10, 30])[0]

doing t-test by color_mode



,color_mode=color,color_mode=grayscale,p-value,significant
slide,,,,
alpha,0.503,0.519,0.176,False
beta,0.478,0.448,0.010,True
gamma,0.563,0.555,0.413,False


### Level 1 or 2?

In [26]:
calculate_t_tests(modl_df, 'level', [1,2], 'color_mode', ['color','grayscale'], 'tubule_area', [1, 10, 30])[0]

doing t-test by level



,level=1,level=2,p-value,significant
slide,,,,
alpha,0.513,0.508,0.709,False
beta,0.457,0.469,0.307,False
gamma,0.543,0.575,0.000,True


### Tubule area 1, 10 or 30?

In [27]:
calculate_t_tests(modl_df, 'tubule_area', [1, 10], 'level', [1,2], 'color_mode', ['color','grayscale'])[0]

doing t-test by tubule_area



,tubule_area=1,tubule_area=10,p-value,significant
slide,,,,
alpha,0.483,0.527,0.004,True
beta,0.443,0.479,0.024,True
gamma,0.561,0.576,0.110,False


In [30]:
calculate_t_tests(modl_df, 'tubule_area', [1, 30], 'level', [1,2], 'color_mode', ['color','grayscale'])[0]

doing t-test by tubule_area



,tubule_area=1,tubule_area=30,p-value,significant
slide,,,,
alpha,0.483,0.521,0.010,True
beta,0.443,0.467,0.095,True
gamma,0.561,0.540,0.087,True


In [29]:
calculate_t_tests(modl_df, 'tubule_area', [10, 30], 'level', [1,2], 'color_mode', ['color','grayscale'])[0]

doing t-test by tubule_area



,tubule_area=10,tubule_area=30,p-value,significant
slide,,,,
alpha,0.527,0.521,0.609,False
beta,0.479,0.467,0.339,False
gamma,0.576,0.540,0.002,True


# Some old experiments on raw data

In [49]:
slides = ["alpha", "beta", "gamma"]
overlaps = [25, 50]

col_postproc=['generation', 'level']
col_model = ['generation', 'level']
for s in slides:
    for o in overlaps:
        col_postproc.append(f"postp iou {s} {o}")
        col_model.append(f"model iou {s} {o}")

## level 1 or 2 on raw model results (tubule area 1)

In [50]:
df[(df['color mode']=='color') & (df['tubule area']==1)][col_model].sort_values(by='level')

,generation,level,model iou alpha 25,model iou alpha 50,model iou beta 25,model iou beta 50,model iou gamma 25,model iou gamma 50
model,,,,,,,,
1:1:color,0,1,0.532,0.542,0.513,0.516,0.592,0.595
1:1:color,1,1,0.500,0.508,0.493,0.498,0.579,0.582
1:1:color,2,1,0.498,0.506,0.499,0.507,0.583,0.586
2:1:color,0,2,0.419,0.427,0.418,0.419,0.544,0.549
2:1:color,1,2,0.420,0.424,0.423,0.429,0.554,0.558
2:1:color,2,2,0.449,0.454,0.421,0.424,0.562,0.563


In [52]:
# level 1 or 2? in raw model outputs, level 1 is better. when overlap is 1 and in color
df[(df['color mode']=='color') & (df['tubule area']==1)][col_postproc].sort_values(by='level')

,generation,level,postp iou alpha 25,postp iou alpha 50,postp iou beta 25,postp iou beta 50,postp iou gamma 25,postp iou gamma 50
model,,,,,,,,
1:1:color,0,1,0.947,0.947,0.773,0.769,0.914,0.914
1:1:color,1,1,0.954,0.955,0.747,0.747,0.912,0.912
1:1:color,2,1,0.943,0.944,0.746,0.748,0.912,0.913
2:1:color,0,2,0.952,0.952,0.637,0.637,0.901,0.901
2:1:color,1,2,0.938,0.940,0.649,0.649,0.898,0.898
2:1:color,2,2,0.952,0.953,0.717,0.711,0.904,0.904


In [51]:
#when overlap is 10, level 2 is better?
df[(df['color mode']=='color') & (df['tubule area']==10)][col_model].sort_values(by='level')

,generation,level,model iou alpha 25,model iou alpha 50,model iou beta 25,model iou beta 50,model iou gamma 25,model iou gamma 50
model,,,,,,,,
1:10:color,0,1,0.522,0.534,0.488,0.493,0.569,0.571
1:10:color,1,1,0.445,0.464,0.462,0.463,0.550,0.559
1:10:color,2,1,0.513,0.523,0.479,0.485,0.580,0.586
2:10:color,0,2,0.551,0.563,0.536,0.540,0.601,0.602
2:10:color,1,2,0.512,0.530,0.540,0.541,0.601,0.602
2:10:color,2,2,0.504,0.521,0.499,0.503,0.583,0.586


In [53]:
#when overlap is 10, level _ is better
df[(df['color mode']=='color') & (df['tubule area']==10)][col_postproc].sort_values(by='level')

,generation,level,postp iou alpha 25,postp iou alpha 50,postp iou beta 25,postp iou beta 50,postp iou gamma 25,postp iou gamma 50
model,,,,,,,,
1:10:color,0,1,0.946,0.946,0.760,0.758,0.912,0.913
1:10:color,1,1,0.942,0.943,0.718,0.696,0.910,0.909
1:10:color,2,1,0.950,0.953,0.772,0.771,0.912,0.913
2:10:color,0,2,0.951,0.950,0.665,0.667,0.908,0.908
2:10:color,1,2,0.945,0.946,0.650,0.650,0.905,0.905
2:10:color,2,2,0.955,0.956,0.685,0.685,0.909,0.910


In [19]:
#when overlap is 30, level neither is better?
df[(df['color mode']=='color') & (df['tubule area']==30)][col_model].sort_values(by='level')

,generation,level,model iou alpha 25,model iou alpha 50,model iou beta 25,model iou beta 50,model iou gamma 25,model iou gamma 50
model,,,,,,,,
1:30:color,0,1,0.558,0.569,0.520,0.523,0.496,0.521
1:30:color,1,1,0.552,0.558,0.457,0.463,0.564,0.564
1:30:color,2,1,0.432,0.443,0.403,0.414,0.417,0.412
2:30:color,0,2,0.528,0.543,0.502,0.504,0.580,0.582
2:30:color,1,2,0.516,0.543,0.459,0.460,0.575,0.577
2:30:color,2,2,0.486,0.509,0.455,0.455,0.566,0.569


In [20]:
# level 1 or 2? in raw model outputs, level ? is better. when overlap is 1 and in grayscale
df[(df['color mode']=='grayscale') & (df['tubule area']==1)][col_model].sort_values(by='level')

,generation,level,model iou alpha 25,model iou alpha 50,model iou beta 25,model iou beta 50,model iou gamma 25,model iou gamma 50
model,,,,,,,,
1:1:grayscale,0,1,0.519,0.524,0.435,0.437,0.538,0.533
1:1:grayscale,1,1,0.415,0.419,0.314,0.317,0.493,0.496
1:1:grayscale,2,1,0.502,0.510,0.404,0.422,0.554,0.560
2:1:grayscale,0,2,0.429,0.432,0.408,0.418,0.520,0.524
2:1:grayscale,1,2,0.602,0.622,0.542,0.540,0.641,0.644
2:1:grayscale,2,2,0.468,0.478,0.417,0.419,0.557,0.562


In [21]:
# level 1 or 2? in raw model outputs, level ? is better. when overlap is 10 and in grayscale
df[(df['color mode']=='grayscale') & (df['tubule area']==10)][col_model].sort_values(by='level')

,generation,level,model iou alpha 25,model iou alpha 50,model iou beta 25,model iou beta 50,model iou gamma 25,model iou gamma 50
model,,,,,,,,
1:10:grayscale,0,1,0.495,0.513,0.423,0.436,0.530,0.541
1:10:grayscale,1,1,0.490,0.501,0.419,0.432,0.533,0.539
1:10:grayscale,2,1,0.574,0.580,0.456,0.465,0.571,0.576
2:10:grayscale,0,2,0.488,0.500,0.392,0.401,0.569,0.572
2:10:grayscale,1,2,0.587,0.592,0.519,0.519,0.599,0.600
2:10:grayscale,2,2,0.572,0.579,0.496,0.499,0.599,0.600


In [ ]:
# 